In [617]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("alltogether2.4v.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]


In [627]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(label_encoder.classes_)

# Normalize features
X_train_normalized = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())



# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation="softmax")
])

['ALU' 'HDPE' 'LDPE' 'PP']


In [591]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_normalized, y_train, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])
# history = model.fit(X_train_normalized, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)




Epoch 1/200
2/2 [==============================] - 1s 137ms/step - loss: 1.4527 - accuracy: 0.2750 - val_loss: 1.4261 - val_accuracy: 0.1935
Epoch 2/200
2/2 [==============================] - 0s 33ms/step - loss: 1.3954 - accuracy: 0.2667 - val_loss: 1.3563 - val_accuracy: 0.3548
Epoch 3/200
2/2 [==============================] - 0s 30ms/step - loss: 1.3510 - accuracy: 0.3250 - val_loss: 1.3046 - val_accuracy: 0.3226
Epoch 4/200
2/2 [==============================] - 0s 28ms/step - loss: 1.3217 - accuracy: 0.2333 - val_loss: 1.2654 - val_accuracy: 0.3226
Epoch 5/200
2/2 [==============================] - 0s 29ms/step - loss: 1.2916 - accuracy: 0.2250 - val_loss: 1.2333 - val_accuracy: 0.3226
Epoch 6/200
2/2 [==============================] - 0s 25ms/step - loss: 1.2699 - accuracy: 0.2500 - val_loss: 1.2058 - val_accuracy: 0.3548
Epoch 7/200
2/2 [==============================] - 0s 25ms/step - loss: 1.2471 - accuracy: 0.3250 - val_loss: 1.1809 - val_accuracy: 0.3548
Epoch 8/200
2/2 [==

In [592]:
# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)



# Convert predictions to percentage likelihood
percentage_likelihood = predictions.max(axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(15):
    if y_test[i] == predicted_labels[i]:
        accuracy = "Correct"
    else:
        accuracy = "False"
    print(f"Sample {i+1}: {accuracy} Predicted {predicted_plastic_types[i]} with {percentage_likelihood[i]:.2f}% likelihood")


# Save the model
model.save("plastic_classifier_model.h5")

total_params = model.count_params()
total_size_bytes = total_params * 4  # Each parameter is usually a 32-bit float
total_size_mb = total_size_bytes / (1024)  # Convert bytes to megabytes

print(f"Total size of the model: {total_size_mb:.2f} KB")


2/2 [==============================] - 0s 3ms/step
Test Accuracy: 0.9210526347160339
Sample 1: Correct Predicted ALU with 99.68% likelihood
Sample 2: Correct Predicted ALU with 99.93% likelihood
Sample 3: Correct Predicted HDPE with 97.15% likelihood
Sample 4: Correct Predicted HDPE with 96.58% likelihood
Sample 5: Correct Predicted LDPE with 54.16% likelihood
Sample 6: Correct Predicted PP with 60.26% likelihood
Sample 7: Correct Predicted HDPE with 94.48% likelihood
Sample 8: False Predicted PP with 56.77% likelihood
Sample 9: Correct Predicted HDPE with 84.14% likelihood
Sample 10: Correct Predicted PP with 80.16% likelihood
Sample 11: Correct Predicted LDPE with 64.69% likelihood
Sample 12: Correct Predicted PP with 62.61% likelihood
Sample 13: Correct Predicted HDPE with 67.23% likelihood
Sample 14: Correct Predicted ALU with 99.91% likelihood
Sample 15: Correct Predicted LDPE with 55.46% likelihood
Total size of the model: 74.52 KB


c:\Users\kylej\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [593]:
# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

# print(top_predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(5):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    print(f"Sample {i+1}:")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][j]:.2f}% likelihood")

# Save the model
model.save("plastic_classifier_model.h5")

2/2 [==============================] - 0s 2ms/step
Test Accuracy: 0.9210526347160339
Sample 1:
  Prediction 1: ALU with 0.00% likelihood
  Prediction 2: LDPE with 0.08% likelihood
  Prediction 3: HDPE with 0.24% likelihood
  Prediction 4: PP with 99.68% likelihood
Sample 2:
  Prediction 1: ALU with 0.00% likelihood
  Prediction 2: LDPE with 0.00% likelihood
  Prediction 3: HDPE with 0.07% likelihood
  Prediction 4: PP with 99.93% likelihood
Sample 3:
  Prediction 1: HDPE with 0.01% likelihood
  Prediction 2: PP with 1.40% likelihood
  Prediction 3: LDPE with 1.44% likelihood
  Prediction 4: ALU with 97.15% likelihood
Sample 4:
  Prediction 1: HDPE with 0.00% likelihood
  Prediction 2: PP with 1.40% likelihood
  Prediction 3: LDPE with 2.02% likelihood
  Prediction 4: ALU with 96.58% likelihood
Sample 5:
  Prediction 1: LDPE with 0.12% likelihood
  Prediction 2: PP with 10.04% likelihood
  Prediction 3: HDPE with 35.68% likelihood
  Prediction 4: ALU with 54.16% likelihood


c:\Users\kylej\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [614]:
# Get manual input from the user
manual_input = np.array([[524161,1534725,2819437,3403538,4467703,2420628,2047009,1183458,860611,1677583,2394933,3207380,2819927,1889676,1374559]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8388608 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 16ms/step
[[0.5601391  0.00938571 0.365221   0.0652542 ]]
0
ALU
Manual Input Prediction:
  Prediction: ALU with 56.01% likelihood
